In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install unidecode
!pip install gensim==4.3.1
!pip install fasttext
!pip install tensorflow

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.9/235.9 kB 4.6 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.8/68.8 kB 3.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Using cached pybind11-2.10.4-py3-none-any.whl (222 kB)
  Created wheel for fasttext: filename=fasttext-0.9.2-cp310-cp310-linux_x86_64.whl size=4393396 sha256=83725e694aa98eb4cfa7dc64d59f80eea7e42698899a3d5096d152d1fc976e26
  Stored in directory: /root/.cache/pip/wheels/a5/13/75/f811c84a8ab36eedbaef977a6a58a98990e8e0f1967f98f394
Successfully built fasttext
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
import gensim
import re
import logging
import numpy as np
import pandas as pd
import multiprocessing

from re import sub
from time import time 
from unidecode import unidecode
from gensim.models import Word2Vec
from collections import defaultdict
from gensim.models import KeyedVectors
from gensim.test.utils import get_tmpfile
from gensim.models.phrases import Phrases, Phraser

import tensorflow as tf
import keras
from keras.models import Sequential
from keras import layers
from tensorflow.keras.optimizers import RMSprop, Adam
from keras.preprocessing.text import Tokenizer
from keras import regularizers
from keras import backend as K
from keras.callbacks import ModelCheckpoint
from keras.utils import pad_sequences, to_categorical
from sklearn.model_selection import train_test_split


In [4]:
#data loading
# we load all the rows(~600k)
colnames=['comment'] 
hiphop = pd.read_csv("/content/drive/MyDrive/hiphopheads.csv", on_bad_lines='skip',engine='python', header = None, names = colnames)
jazz = pd.read_csv("/content/drive/MyDrive/jazz.csv", on_bad_lines='skip',engine='python', header = None, names = colnames)
punk = pd.read_csv("/content/drive/MyDrive/punk.csv", on_bad_lines='skip',engine='python', header = None, names = colnames)
metal = pd.read_csv("/content/drive/MyDrive/metal.csv", on_bad_lines='skip',engine='python', header = None, names = colnames)
classical = pd.read_csv("/content/drive/MyDrive/classicalmusic.csv", on_bad_lines='skip',engine='python', header = None, names = colnames)

In [5]:
fileList2 = []
#fileList2.append(hiphop.head(30000))
fileList2.append(jazz.sample(2000))
#fileList2.append(punk.head(10000))
fileList2.append(metal.sample(2000))
#fileList2.append(classical.head(250))

In [6]:
merge = pd.concat(fileList2)

merge = merge[merge['comment'].str.len()<500]

fileList = []
fileList.append(merge)

print(type(merge))
print(merge)

<class 'pandas.core.frame.DataFrame'>
                                                       comment
t1_iq5qpe0   Joe Pass, Charlie Christian, Jim Hall, Django ...
t1_ijtlk83                            Willowcrest. Buddy Rich.
t1_iowxcdr   Bossa Nova is some of the greatest music creat...
t1_jbc2t74    I can hear this sounding pretty cool with a band
t1_jevbgd4   Bill Evans Black Forest session ranks right up...
...                                                        ...
*I am a bot   and this action was performed automatically. ...
t1_i3dqtgk   Yeah it becomes less and less noticable the lo...
t1_iwnt4cx   my favorite Avantasia albums are probably the ...
t1_hzq000w   As soon as I get the opportunity. I won't hesi...
t1_j6ssr3y   Caught the first few songs of his set when he ...

[3123 rows x 1 columns]


In [7]:
def text_to_word_list(text, remove_polish_letters):
    ''' Pre process and convert texts to a list of words 
    method inspired by method from eliorc github repo: https://github.com/eliorc/Medium/blob/master/MaLSTM.ipynb'''
    text = remove_polish_letters(text)
    text = str(text)
    text = text.lower()

    # Clean the text
    text = sub(r"[^A-Za-z0-9^,!?.\/'+]", " ", text)
    text = sub(r"\+", "", text)
    text = sub(r",", "", text)
    text = sub(r"\.", "", text)
    text = sub(r"!", "", text)
    text = sub(r"\?", "", text)
    text = sub(r"'", "", text)
    text = sub(r":", "", text)
    text = sub(r"\s{2,}", " ", text)

    text = text.split()

    return text  

In [8]:
from sklearn.cluster import KMeans
from IPython.display import display
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, f1_score

In [9]:
def create_tfidf_dictionary(x, transformed_file, features):
    '''
    create dictionary for each input sentence x, where each word has assigned its tfidf score
    
    x - row of dataframe, containing sentences, and their indexes,
    transformed_file - all sentences transformed with TfidfVectorizer
    features - names of all words in corpus used in TfidfVectorizer

    '''
    vector_coo = transformed_file[x.name].tocoo()
    vector_coo.col = features.iloc[vector_coo.col].values
    dict_from_coo = dict(zip(vector_coo.col, vector_coo.data))
    return dict_from_coo

def replace_tfidf_words(x, transformed_file, features):
    '''
    replacing each word with it's calculated tfidf dictionary with scores of each word
    x - row of dataframe, containing sentences, and their indexes,
    transformed_file - all sentences transformed with TfidfVectorizer
    features - names of all words in corpus used in TfidfVectorizer
    '''
    dictionary = create_tfidf_dictionary(x, transformed_file, features)   
    return list(map(lambda y:dictionary[f'{y}'], x.title.split()))

In [10]:
def replace_sentiment_words(word, sentiment_dict):
    '''
    replacing each word with its associated sentiment score from sentiment dict
    '''
    try:
        out = sentiment_dict[word]
    except KeyError:
        out = 0
    return out

In [11]:
predictedList = []

In [12]:
def divide_to_parts(l, n):
    for i in range(0, len(l), n):
        yield l[i:i + n]


for file_ in fileList:
  parts = divide_to_parts(file_.comment, 3000)    
  file_['rate'] = 1
  file_cleaned = file_.dropna().drop_duplicates().reset_index(drop=True).rename(columns={'comment':'title'})
  file_cleaned = file_cleaned[file_cleaned.rate!=0]
  file_cleaned.rate.value_counts()/len(file_cleaned)

  file_cleaned.title = file_cleaned.title.apply(lambda x: text_to_word_list(x, unidecode))
  file_model = file_cleaned.copy()
  file_model = file_model[file_model.title.str.len()>1]


  parts = divide_to_parts(file_model.title, 3000)
  for part in parts:    
    #Detect bigrams with gensim's Phraser module.
    sent = [row for row in part]
    phrases = Phrases(sent, min_count=1, progress_per=50000)
    bigram = Phraser(phrases)
    sentences = bigram[sent]

    #word embeddings with CBOW word2vec algorithm found in gensim module
    w2v_model = Word2Vec(min_count=3,
                      window=4,
                      vector_size=300,
                      sample=1e-5, 
                      alpha=0.03, 
                      min_alpha=0.0007, 
                      negative=20,
                      workers=multiprocessing.cpu_count()-1)

    start = time()

    w2v_model.build_vocab(sentences, progress_per=50000)

    print('Time to build vocab: {} mins'.format(round((time() - start) / 60, 2)))

    start = time()

    w2v_model.train(sentences,
                    total_examples=w2v_model.corpus_count,
                    epochs=30,
                    report_delay=1)

    print('Time to train the model: {} mins'.format(round((time() - start) / 60, 2)))

    w2v_model.init_sims(replace=True)


    w2v_model.save("word2vec.model")
    
    file_export = file_model.copy()
    file_export['old_title'] = file_export.title
    file_export.old_title = file_export.old_title.str.join(' ')
    file_export.title = file_export.title.apply(lambda x: ' '.join(bigram[x]))
    file_export.rate = file_export.rate.astype('int8')

    file_export[['title', 'rate']].to_csv('cleaned_dataset.csv', index=False)

    # KMeans clusters for sentiment


    word_vectors = Word2Vec.load("word2vec.model").wv

    model = KMeans(n_clusters=2,
                  max_iter=1000,
                  random_state=42,
                  n_init=50)
    model.fit(X=word_vectors.vectors.astype('double'))

    word_vectors.similar_by_vector(model.cluster_centers_[1], topn=10, restrict_vocab=None)

    positive_cluster_index = 1
    positive_cluster_center = model.cluster_centers_[positive_cluster_index]
    negative_cluster_center = model.cluster_centers_[1-positive_cluster_index]

    words = pd.DataFrame(word_vectors.index_to_key)
    words.columns = ['words']
    words['vectors'] = words.words.apply(lambda x: word_vectors[f'{x}'])
    words['cluster'] = words.vectors.apply(lambda x: model.predict([np.array(x)]))
    words.cluster = words.cluster.apply(lambda x: x[0])
    words['cluster_value'] = [1 if i==positive_cluster_index else -1 for i in words.cluster]
    words['closeness_score'] = words.apply(lambda x: 1/(model.transform([x.vectors]).min()), axis=1)
    words['sentiment_coeff'] = words.closeness_score * words.cluster_value

    words[['words', 'sentiment_coeff']].to_csv('sentiment_dictionary.csv', index=False)

    #Labelling data

    final_file = pd.read_csv('cleaned_dataset.csv')
    sentiment_map = pd.read_csv('sentiment_dictionary.csv')
    sentiment_dict = dict(zip(sentiment_map.words.values, sentiment_map.sentiment_coeff.values))

    file_weighting = final_file.copy()
    tfidf = TfidfVectorizer(tokenizer=lambda y: y.split(), norm=None)
    tfidf.fit(file_weighting.title)
    features = pd.Series(tfidf.get_feature_names_out())
    transformed = tfidf.transform(file_weighting.title)

    replaced_tfidf_scores = file_weighting.apply(lambda x: replace_tfidf_words(x, transformed, features), axis=1)

    replaced_closeness_scores = file_weighting.title.apply(lambda x: list(map(lambda y: replace_sentiment_words(y, sentiment_dict), x.split())))

    replacement_df = pd.DataFrame(data=[replaced_closeness_scores, replaced_tfidf_scores, file_weighting.title, file_weighting.rate]).T
    replacement_df.columns = ['sentiment_coeff', 'tfidf_scores', 'sentence', 'sentiment']
    replacement_df['sentiment_rate'] = replacement_df.apply(lambda x: np.array(x.loc['sentiment_coeff']) @ np.array(x.loc['tfidf_scores']), axis=1)
    replacement_df['prediction'] = (replacement_df.sentiment_rate>0).astype('int8')
    replacement_df['sentiment'] = [1 if i==1 else 0 for i in replacement_df.sentiment]

    predictedList.append(replacement_df)




Time to build vocab: 0.0 mins


<ipython-input-12-4f3d5e98d49c>:51: DeprecationWarning: Call to deprecated `init_sims` (Gensim 4.0.0 implemented internal optimizations that make calls to init_sims() unnecessary. init_sims() is now obsoleted and will be completely removed in future versions. See https://github.com/RaRe-Technologies/gensim/wiki/Migrating-from-Gensim-3.x-to-4).
  w2v_model.init_sims(replace=True)


Time to train the model: 0.1 mins


/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [13]:
merge = pd.concat(predictedList)

tempList = []
tempList.append(merge)

In [14]:
listTwoElems = []

for elem in tempList:
  listTwoElems.append(elem[['sentence', 'prediction']])


In [15]:
print(listTwoElems[0])

                                               sentence  prediction
0     joe_pass charlie christian jim_hall django_rei...           1
1                                willowcrest buddy_rich           1
2     bossa_nova is some of the_greatest music creat...           1
3      i can_hear this sounding pretty_cool with a band           1
4     bill_evans black forest session ranks right up...           1
...                                                 ...         ...
2749                  yield is_such an_underrated album           0
2750  yeah it becomes less and less noticable the lo...           1
2751  my_favorite avantasia albums are probably the ...           1
2752   as soon as i get the opportunity i wont hesitate           0
2753  caught the first few_songs of his set when_he ...           1

[2754 rows x 2 columns]


In [16]:
with pd.option_context('display.max_colwidth', None):
  display(listTwoElems[0])

,sentence,prediction
0,joe_pass charlie christian jim_hall django_reinhardt,1
1,willowcrest buddy_rich,1
2,bossa_nova is some of the_greatest music created in the_past 100 years,1
3,i can_hear this sounding pretty_cool with a band,1
4,bill_evans black forest session ranks right up_there with the_village vanguard recordings for_me,1
...,...,...
2749,yield is_such an_underrated album,0
2750,yeah it becomes less and less noticable the longer you listen_to the album haha i completely missed that they had_another album out after garden_of storms i need_to go check that out now,1
2751,my_favorite avantasia albums are probably the two parts_of this,1
2752,as soon as i get the opportunity i wont hesitate,0


Hugging Face Zero-shot

In [17]:
!pip install transformers flair

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 47.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 373.1/373.1 kB 32.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 27.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 113.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 788.5/788.5 kB 68.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.6/135.6 kB 16.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 71.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.1/53.1 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.7/19.7 MB 82.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting re

In [40]:

from flair.models import TextClassifier
classifier = TextClassifier.load("en-sentiment")

from flair.data import Sentence

from sklearn.metrics import accuracy_score

In [41]:
my_df = listTwoElems[0]
my_df.rename(columns={'prediction':'label'}, inplace=True)
my_df.rename(columns={'sentence':'review'}, inplace=True)
my_df.sample(5)

<ipython-input-41-abbfef69b5ef>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  my_df.rename(columns={'prediction':'label'}, inplace=True)
<ipython-input-41-abbfef69b5ef>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  my_df.rename(columns={'sentence':'review'}, inplace=True)


,review,label
797,etta jones,1
614,the swedish band se quartet,1
2056,its on my check this list so_far so_good,1
1897,i had forgotten_about them,1
2443,hasnt given much attention to the mercyful_fat...,1


In [42]:
my_df['review'].value_counts()

thank_you                                                                                                                                                                                                                                                                                               7
hell_yeah                                                                                                                                                                                                                                                                                               2
this is great                                                                                                                                                                                                                                                                                           2
great album                                                                                               

In [43]:
my_df['label'].value_counts()

1    2179
0     575
Name: label, dtype: int64

In [44]:
from transformers import pipeline

In [45]:
classifier_hf = pipeline(task='zero-shot-classification',
                         model='facebook/bart-large-mnli',
                         device=0)

In [46]:
sequences = my_df['review'].to_list()
candidates = ['positive', 'negative']
hyp_template = "The predicted sentiment is {}"

predictions_hf = classifier_hf(sequences, candidates, hypothesis_template=hyp_template)

predictions_hf = pd.DataFrame(predictions_hf)
predictions_hf.head()

,sequence,labels,scores
0,joe_pass charlie christian jim_hall django_rei...,"[positive, negative]","[0.5638939738273621, 0.4361060857772827]"
1,willowcrest buddy_rich,"[positive, negative]","[0.9258539080619812, 0.0741460770368576]"
2,bossa_nova is some of the_greatest music creat...,"[positive, negative]","[0.8844901323318481, 0.11550989747047424]"
3,i can_hear this sounding pretty_cool with a band,"[positive, negative]","[0.9968761801719666, 0.0031238326337188482]"
4,bill_evans black forest session ranks right up...,"[positive, negative]","[0.8634577989578247, 0.13654223084449768]"


In [47]:
predictions_hf.head()

,sequence,labels,scores
0,joe_pass charlie christian jim_hall django_rei...,"[positive, negative]","[0.5638939738273621, 0.4361060857772827]"
1,willowcrest buddy_rich,"[positive, negative]","[0.9258539080619812, 0.0741460770368576]"
2,bossa_nova is some of the_greatest music creat...,"[positive, negative]","[0.8844901323318481, 0.11550989747047424]"
3,i can_hear this sounding pretty_cool with a band,"[positive, negative]","[0.9968761801719666, 0.0031238326337188482]"
4,bill_evans black forest session ranks right up...,"[positive, negative]","[0.8634577989578247, 0.13654223084449768]"


In [48]:
predictions_hf['predicted_label'] = predictions_hf['labels'].apply(lambda x: x[0])
predictions_hf['predicted_label'] = predictions_hf['predicted_label'].map({'positive' : 1, 'negative' : 0})
predictions_hf['predicted_score'] = predictions_hf['scores'].apply(lambda x: x[0])

predictions_hf['ground_truth'] = my_df['label']

predictions_hf.drop(['labels', 'scores'], axis=1, inplace=True)


predictions_hf.head()

,sequence,predicted_label,predicted_score,ground_truth
0,joe_pass charlie christian jim_hall django_rei...,1,0.563894,1
1,willowcrest buddy_rich,1,0.925854,1
2,bossa_nova is some of the_greatest music creat...,1,0.884490,1
3,i can_hear this sounding pretty_cool with a band,1,0.996876,1
4,bill_evans black forest session ranks right up...,1,0.863458,1


Accuracy is fairly LOW, but I beleive that is due to small sample of data. Colab wouldn't let me use more of it.

In [49]:
accuracy_score(predictions_hf['predicted_label'], predictions_hf['ground_truth'])

0.5751633986928104